<a href="https://colab.research.google.com/github/GGlivePh/BSF_Phenotyping_pipelines/blob/main/Segmentation%2C%20feature%20extraction%20and%20weight%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/GGlivePh/BSF_Phenotyping_pipelines.git

Cloning into 'BSF_Phenotyping_pipelines'...
remote: Enumerating objects: 2797, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 2797 (delta 7), reused 0 (delta 0), pack-reused 2777 (from 1)
Receiving objects: 100% (2797/2797), 261.72 MiB | 19.40 MiB/s, done.
Resolving deltas: 100% (14/14), done.
Updating files: 100% (2215/2215), done.
Filtering content: 100% (2199/2199), 716.46 MiB | 16.78 MiB/s, done.


In [2]:
import os
os.chdir('BSF_Phenotyping_pipelines')

In [3]:
import numpy as np
import pandas as pd
import cv2

In [4]:
#!pip install ultralytics==8.3.124


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [8]:
#Load the yolov8 trained segmentation model

from ultralytics import YOLO
model = YOLO("models/YoloRetrain.pt")


In [7]:
#Load Images for larvae segmentation and save the result
import glob

image_paths = glob.glob("models/input-images/*.jpg")  # Adjust path
results = model(image_paths, save=True, project="results/output",
    name="bsf_segment_results")



0: 480x640 1 larva, 2836.4ms
Speed: 42.4ms preprocess, 2836.4ms inference, 40.4ms postprocess per image at shape (1, 3, 480, 640)
Results saved to results/output/bsf_segment_results


In [9]:
# Code to extract the features
# Loop through each image result
# Data storage
data = []

for i, result in enumerate(results):
    image_name = image_paths[i].split("/")[-1]

    if result.masks is None:
        continue  # No detections

    masks = result.masks.data.cpu().numpy()  # shape: [n, H, W]

    for j, mask in enumerate(masks):
        # Convert float mask to binary image (uint8)
        binary_mask = (mask > 0.5).astype(np.uint8) * 255

        # Find contours
        contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Compute area and perimeter
        for contour in contours:
            area = cv2.contourArea(contour)
            perimeter = cv2.arcLength(contour, closed=True)
            data.append({
                'image': image_name,
                'mask_id': j,
                'area': area,
                'perimeter': perimeter
            })

# Convert to DataFrame and save as CSV
df = pd.DataFrame(data)
df.to_csv("models/mask_metrics.csv", index=False)
print("Saved to mask_metrics.csv")

Saved to mask_metrics.csv


In [ ]:
!pip uninstall opencv-python
!pip install opencv-contrib-python


Found existing installation: opencv-python 4.11.0.86
Uninstalling opencv-python-4.11.0.86:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/cv2/*
    /usr/local/lib/python3.11/dist-packages/opencv_python-4.11.0.86.dist-info/*
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libQt5Core-e7f476e2.so.5.15.16
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libQt5Gui-1ce8cbfe.so.5.15.16
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libQt5Test-9ac3ed15.so.5.15.16
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libQt5Widgets-cd430389.so.5.15.16
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libQt5XcbQpa-b18b5a78.so.5.15.16
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libX11-xcb-0e257303.so.1.0.0
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libXau-00ec42fe.so.6.0.0
    /usr/local/lib/python3.11/dist-packages/opencv_python.libs/libavcodec-76c43bf0.so.59.37.100
    /usr/loca

In [ ]:
#pip install fil_finder astropy scikit-image opencv-python-headless

import os
import cv2
import numpy as np
import pandas as pd
import torch
import math
data = []

import os
import math
import cv2
import numpy as np
import pandas as pd

data = []
Calibration_factor = 0.0119 #Pixel to mm

def calculate_length_width(perimeter, area):
    # Intermediate calculation
    inner_sqrt = (perimeter ** 2) / 4 - area
    if inner_sqrt < 0:
        raise ValueError("Invalid inputs: square root of a negative number.")

    sqrt_value = math.sqrt(inner_sqrt)

    # Length and Width formulas
    length = abs(perimeter / 4 + sqrt_value)*Calibration_factor
    width = abs(perimeter / 4 - sqrt_value)*Calibration_factor

    return length, width

for i, result in enumerate(results):
    image_name = os.path.basename(image_paths[i])

    if result.masks is None:
        continue  # Skip if no masks

    masks = result.masks.data.cpu().numpy()  # [n, H, W]

    for j, mask in enumerate(masks):
        binary_mask = (mask > 0.5).astype(np.uint8) * 255

        # Resize for analysis
        resized_mask = cv2.resize(binary_mask, None, fx=6.3375, fy=6.3375)

        # --------- AREA & PERIMETER ---------
        contours, _ = cv2.findContours(resized_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(contours) == 0:
            continue

        area = cv2.contourArea(contours[0])
        perimeter = cv2.arcLength(contours[0], closed=True)

        # --------- LENGTH & WIDTH ---------
        try:
            length, width = calculate_length_width(perimeter, area)
        except ValueError:
            length, width = None, None  # Optional: log or skip if desired

        # --------- Append all metrics ---------
        data.append({
            'image': image_name,
            'mask_id': j,
            'area': area,
            'perimeter': perimeter,
            'length': length,
            'width': width
        })

# Save to CSV
df = pd.DataFrame(data)
output_csv = "models/larva_metrics.csv"
df.to_csv(output_csv, index=False)
print(f"Saved larva metrics to: {output_csv}")



Saved larva metrics to: /content/drive/MyDrive/BSF_Phenotyping_pipelines/models/larva_metrics.csv


In [ ]:
#For weight prediction
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import math
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import make_scorer

# Load the Larva dataset
df = pd.read_csv(r"AllImgfeatures_YoloRetrainmodel.csv")

# Split the data into features and labels
df.describe()

#df.isnull().sum()
#df.dropna(axis=0, inplace=True)  # axis=0 for rows, axis=1 for columns

X =  np.array(df[['Area', 'Length_Actual', 'Wid2']])
#X =  np.array(df[['Area', 'Length']])
#print(X)
y = df['Weight']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Define cross-validation strategy (5-fold CV)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize models
LR = LinearRegression()


# Train LR using K-Fold CV (No hyperparameter tuning)
rmse_lr_scores = np.sqrt(-cross_val_score(LR, X_train, y_train, scoring='neg_mean_squared_error', cv=kf))
r2_lr_scores = cross_val_score(LR, X_train, y_train, scoring='r2', cv=kf)

# Train final models on the full training set
LR.fit(X_train, y_train)

# Predictions on test set
y_pred_lr = LR.predict(X_test)

# Compute RMSE and R² for each model on the test set
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
r2_lr = r2_score(y_test, y_pred_lr)

# Compute mean and standard deviation for all models
lr_rmse_mean, lr_rmse_std = rmse_lr_scores.mean(), rmse_lr_scores.std()
lr_r2_mean, lr_r2_std = r2_lr_scores.mean(), r2_lr_scores.std()

# Print results
print(f"Linear Regression - RMSE: {rmse_lr:.3f} ± {lr_rmse_std:.2f}, R²: {r2_lr:.2f} ± {lr_r2_std:.2f}")


LinearRegression()

In [ ]:
import joblib

# Save the model
joblib.dump(LR, "models/linear_regression_model.pkl")



['/content/drive/MyDrive/BSF_Phenotyping_pipelines/models/linear_regression_model.pkl']

In [ ]:

# Load the model
model = joblib.load("models/linear_regression_model.pkl")

csv_path = "models/larva_metrics.csv"
df = pd.read_csv(csv_path)

# === 3. Extract features for prediction ===
# Ensure these exact columns exist in your CSV
feature_columns = ['area', 'length', 'width']
X_new = df[feature_columns].values

# === 4. Predict weights ===
predicted_weights = model.predict(X_new)

# === 5. Add predictions to the DataFrame ===
df['Predicted_Weight'] = predicted_weights

# === 6. Save to a new CSV (optional) ===
output_csv = "larva_with_predicted_weights.csv"
df.to_csv(output_csv, index=False)

print(f"Predictions saved to: {output_csv}")


Predictions saved to: /content/drive/MyDrive/BSF_Phenotyping_pipelines/larva_with_predicted_weights.csv
